## Imports

In [1]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate

## EDA

In [2]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [3]:
link = pd.read_csv('./ml-latest-small/links.csv')

In [4]:
rating = pd.read_csv('./ml-latest-small/ratings.csv')

In [5]:
tags = pd.read_csv('./ml-latest-small/tags.csv')

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
movies.title.value_counts()

Confessions of a Dangerous Mind (2002)                                     2
Saturn 3 (1980)                                                            2
Eros (2004)                                                                2
Emma (1996)                                                                2
War of the Worlds (2005)                                                   2
                                                                          ..
Flickering Lights (Blinkende lygter) (2000)                                1
Yards, The (2000)                                                          1
Lovers of the Arctic Circle, The (Los Amantes del Círculo Polar) (1998)    1
Robot Carnival (Roboto kânibauru) (1987)                                   1
Sex Ed (2014)                                                              1
Name: title, Length: 9737, dtype: int64

In [9]:
#Check to see if there are any duplicate titles
movies.title.duplicated().sum()

5

In [10]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [11]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [12]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


In [13]:
#Check length of DataFrame

len(movies)

9737

In [14]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

In [15]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [16]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [17]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [18]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [19]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [20]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [21]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

In [22]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [23]:
rating.duplicated().sum()

0

In [24]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [25]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [26]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [27]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100849,64997,NaN,NaN,NaN,68.0,2.5,2008-12-28 20:55:15
100850,144606,NaN,NaN,NaN,111.0,4.0,2018-01-31 23:27:37
100851,147002,NaN,NaN,NaN,318.0,4.0,2017-08-08 15:45:52
100852,26958,NaN,NaN,NaN,509.0,3.5,2015-07-04 17:42:33


In [28]:
movie_rating.dropna(inplace=True)

In [29]:
movie_rating['genres']=[row.strip().lower().replace('|',',') for row in movie_rating['genres']]

In [30]:
movie_rating['userId'].nunique()

610

In [31]:
stats = movie_rating.filter(['rating', 'timestamp']).describe()
stats

,rating
count,100813.000000
mean,3.501557
std,1.042494
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


In [32]:
# print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
# print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

In [33]:
movie_rating['Title'] = movie_rating['Title'].str.split('(', n=1, expand=True)[0]

movie_rating['year_released'] = movie_rating['Title'].str.split('(', n=1, expand=True)[0]

In [35]:
# Drop original column title
#movie_rating.drop('title',axis=1,inplace=True)

In [36]:
# Format year_released column

movie_rating['year_released'] = movie_rating.year_released.str.replace(')', '')

In [37]:
#converted timestamp to datetime
movie_rating['timestamp'] = pd.to_datetime(movie_rating['timestamp'], unit='s')

In [38]:
movie_rating['genres'] = [row.strip().lower().replace('|',',') for row in movie_rating['genres']]

In [39]:
movie_rating['genres'] = movie_rating['genres'].to_list()

In [40]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,"adventure,animation,children,comedy,fantasy",Toy Story,Toy Story,1.0,4.0,2000-07-30 18:45:03
1,1,"adventure,animation,children,comedy,fantasy",Toy Story,Toy Story,5.0,4.0,1996-11-08 06:36:02
2,1,"adventure,animation,children,comedy,fantasy",Toy Story,Toy Story,7.0,4.5,2005-01-25 06:52:26
3,1,"adventure,animation,children,comedy,fantasy",Toy Story,Toy Story,15.0,2.5,2017-11-13 12:59:30
4,1,"adventure,animation,children,comedy,fantasy",Toy Story,Toy Story,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,"action,animation,comedy,fantasy",Black Butler: Book of the Atlantic,Black Butler: Book of the Atlantic,184.0,4.0,2018-09-16 14:44:42
100844,193583,"animation,comedy,fantasy",No Game No Life: Zero,No Game No Life: Zero,184.0,3.5,2018-09-16 14:52:25
100845,193585,drama,Flint,Flint,184.0,3.5,2018-09-16 14:56:45
100846,193587,"action,animation",Bungo Stray Dogs: Dead Apple,Bungo Stray Dogs: Dead Apple,184.0,3.5,2018-09-16 15:00:21


## Split

In [41]:
train, test = train_test_split(movie_rating)

In [42]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [ ]:
test_data = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader)

## Tuning

### KNNBasics

In [43]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.9s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   26.0s finished


In [44]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 5}, 'mae': {'k': 10, 'min_k': 5}}

In [45]:
base_model.best_estimator

{'rmse': <surprise.prediction_algorithms.knns.KNNBasic at 0x7f81425dc850>,
 'mae': <surprise.prediction_algorithms.knns.KNNBasic at 0x7f813046ee80>}

In [46]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [79]:
#instantiate KNN model 
model1=KNNBasic(k=10, min_k=5)

In [80]:
#instaniatite and perform cross val with training data
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9563  0.9488  0.9518  0.9621  0.9547  0.9547  0.0045  
MAE (testset)     0.7317  0.7294  0.7305  0.7332  0.7285  0.7307  0.0017  
Fit time          0.04    0.04    0.04    0.04    0.04    0.04    0.00    
Test time         0.39    0.43    0.39    0.44    0.40    0.41    0.02    


{'test_rmse': array([0.95631534, 0.94876981, 0.95179937, 0.96212021, 0.95472984]),
 'test_mae': array([0.7316972 , 0.72943036, 0.73046261, 0.73324337, 0.72850471]),
 'fit_time': (0.03758716583251953,
  0.03916811943054199,
  0.040289878845214844,
  0.03896689414978027,
  0.039347171783447266),
 'test_time': (0.3893759250640869,
  0.43082094192504883,
  0.3918931484222412,
  0.4392368793487549,
  0.3990318775177002)}

In [81]:
#use model to predict
model1.predict(uid=10, iid = 20)
#this model just predicted what user 10 would predict movie 20

Prediction(uid=10, iid=20, r_ui=None, est=2.452185559802743, details={'actual_k': 8, 'was_impossible': False})

looking at the output above we can determine that our model predicted that use 10 would rate movie 20 a 2.5 give or take our RSME which is .95.

## Models

### SVD

In [50]:
model2 = SVD()

In [51]:
model2

In [52]:
cross_validate(model2, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8799  0.8823  0.8792  0.8888  0.8793  0.8819  0.0036  
MAE (testset)     0.6769  0.6812  0.6787  0.6820  0.6760  0.6790  0.0024  
Fit time          2.99    2.96    2.97    2.91    2.95    2.96    0.03    
Test time         0.10    0.06    0.10    0.06    0.10    0.09    0.02    


{'test_rmse': array([0.87986737, 0.88229829, 0.87922955, 0.88877772, 0.87925128]),
 'test_mae': array([0.6768559 , 0.68121776, 0.67866216, 0.68203205, 0.67599941]),
 'fit_time': (2.9876179695129395,
  2.955847978591919,
  2.9683547019958496,
  2.9087958335876465,
  2.9544639587402344),
 'test_time': (0.10031986236572266,
  0.06319904327392578,
  0.10327601432800293,
  0.06260204315185547,
  0.10175490379333496)}

In [53]:
param_grid = {'n_factors':[10,20,50],'n_epochs': [5, 10, 15], 'lr_all': [0.002,0.005,0.01],
             'reg_all': [0.2,0.4,0.6]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:  4.6min finished


In [54]:
gs_model2.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2},
 'mae': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}}

In [55]:
cross_validate(SVD(n_factors=10,n_epochs=15,lr_all=0.01,reg_all=0.2), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8703  0.8829  0.8810  0.8693  0.8789  0.8765  0.0056  
MAE (testset)     0.6729  0.6794  0.6789  0.6705  0.6751  0.6753  0.0034  
Fit time          0.64    0.63    0.63    0.62    0.64    0.63    0.01    
Test time         0.06    0.15    0.06    0.10    0.06    0.09    0.03    


{'test_rmse': array([0.87033471, 0.88287097, 0.88102844, 0.86929909, 0.87894888]),
 'test_mae': array([0.67285101, 0.67936439, 0.67890662, 0.67046293, 0.67508255]),
 'fit_time': (0.6370811462402344,
  0.6254432201385498,
  0.6267590522766113,
  0.6171550750732422,
  0.6358151435852051),
 'test_time': (0.06285309791564941,
  0.14760279655456543,
  0.06144309043884277,
  0.10498213768005371,
  0.06133294105529785)}

In [56]:
param_grid = {'n_factors':[5,10,15],'n_epochs': [15, 30, 45], 'lr_all': [0.01,0.05,0.1],
             'reg_all': [0.1,0.2,0.3]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:  8.5min finished


In [57]:
gs_model2.best_params

{'rmse': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1},
 'mae': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1}}

In [58]:
cross_validate(SVD(n_factors=5,n_epochs=15,lr_all=0.01,reg_all=0.1), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8641  0.8745  0.8739  0.8809  0.8774  0.8742  0.0056  
MAE (testset)     0.6639  0.6692  0.6741  0.6789  0.6755  0.6723  0.0052  
Fit time          0.51    0.49    0.48    0.49    0.49    0.49    0.01    
Test time         0.06    0.10    0.06    0.10    0.06    0.08    0.02    


{'test_rmse': array([0.86413918, 0.87445181, 0.8738758 , 0.88091212, 0.87740597]),
 'test_mae': array([0.66389492, 0.66920538, 0.67414984, 0.67887745, 0.67550934]),
 'fit_time': (0.5105800628662109,
  0.49085426330566406,
  0.4837930202484131,
  0.4881889820098877,
  0.48850297927856445),
 'test_time': (0.06117367744445801,
  0.10461688041687012,
  0.06062602996826172,
  0.10467982292175293,
  0.060652971267700195)}

### NMF

In [59]:
model3 = NMF()

In [60]:
cross_validate(model3, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9363  0.9326  0.9467  0.9490  0.9411  0.9411  0.0061  
MAE (testset)     0.7176  0.7152  0.7246  0.7284  0.7242  0.7220  0.0049  
Fit time          3.05    2.98    2.94    2.96    2.99    2.98    0.04    
Test time         0.05    0.10    0.05    0.09    0.05    0.07    0.02    


{'test_rmse': array([0.93633888, 0.93256234, 0.94673292, 0.94895535, 0.94111028]),
 'test_mae': array([0.71764466, 0.71518121, 0.72456158, 0.72842558, 0.72416211]),
 'fit_time': (3.0503530502319336,
  2.9818029403686523,
  2.936337947845459,
  2.963634967803955,
  2.988227128982544),
 'test_time': (0.0538487434387207,
  0.09950017929077148,
  0.05143594741821289,
  0.09483599662780762,
  0.052422285079956055)}

In [61]:
param_grid = {'n_factors':[10,15,20],'n_epochs': [25, 50, 75]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.3min finished


In [62]:
gs_model3.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 50},
 'mae': {'n_factors': 20, 'n_epochs': 50}}

In [63]:
cross_validate(NMF(n_factors=10,n_epochs=25), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9787  0.9718  0.9641  0.9581  0.9698  0.9685  0.0070  
MAE (testset)     0.7384  0.7337  0.7293  0.7294  0.7332  0.7328  0.0034  
Fit time          1.27    1.26    1.27    1.25    1.26    1.26    0.01    
Test time         0.05    0.10    0.05    0.05    0.05    0.06    0.02    


{'test_rmse': array([0.97871336, 0.97183541, 0.96406206, 0.95810664, 0.96982801]),
 'test_mae': array([0.73840637, 0.73371541, 0.7292894 , 0.72940379, 0.73315057]),
 'fit_time': (1.2730309963226318,
  1.2585961818695068,
  1.2719230651855469,
  1.2542121410369873,
  1.2580420970916748),
 'test_time': (0.05244708061218262,
  0.10116100311279297,
  0.05187511444091797,
  0.051885128021240234,
  0.05135083198547363)}

In [64]:
param_grid = {'n_factors':[3,5,10],'n_epochs': [15, 20, 25]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   40.4s finished


In [65]:
gs_model3.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 25},
 'mae': {'n_factors': 10, 'n_epochs': 25}}

## Deployment 

In [82]:
train['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(train.userId[10], x).est)
train.sort_values(by='est_rating_user10', ascending=False, inplace=True)

In [83]:
train

,movieId,genres,Title,year_released,userId,rating,timestamp,est_rating_user10
54435,3451,drama,Guess Who's Coming to Dinner,Guess Who's Coming to Dinner,140.0,4.0,2002-05-20 12:49:58,5.0
24202,1148,"animation,children,comedy,crime",Wallace & Gromit: The Wrong Trousers,Wallace & Gromit: The Wrong Trousers,477.0,4.5,2008-01-21 18:16:17,5.0
24205,1148,"animation,children,comedy,crime",Wallace & Gromit: The Wrong Trousers,Wallace & Gromit: The Wrong Trousers,502.0,4.5,2005-03-25 13:28:31,5.0
24188,1148,"animation,children,comedy,crime",Wallace & Gromit: The Wrong Trousers,Wallace & Gromit: The Wrong Trousers,310.0,5.0,2004-03-07 08:57:36,5.0
24177,1148,"animation,children,comedy,crime",Wallace & Gromit: The Wrong Trousers,Wallace & Gromit: The Wrong Trousers,186.0,4.0,2002-09-03 19:20:31,5.0
...,...,...,...,...,...,...,...,...
58646,3973,"crime,horror,mystery,thriller",Book of Shadows: Blair Witch 2,Book of Shadows: Blair Witch 2,599.0,0.5,2018-02-22 21:45:33,1.0
67815,5560,"comedy,musical",À nous la liberté,À nous la liberté,527.0,1.0,2002-09-28 00:39:41,1.0
61610,4317,"comedy,romance",Love Potion #9,Love Potion #9,307.0,1.0,2007-08-03 20:43:26,1.0
60286,4125,"comedy,sci-fi",Leonard Part 6,Leonard Part 6,414.0,1.0,2001-05-29 14:09:43,1.0


In [69]:
user = input('userId: ')
#genre = input('What genres are you interested in? ')
#num_recs = input('How many recomendations would you like?')

userId: 10


In [75]:
train['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(uid=user, iid=x).est)
train.sort_values(by='est_rating_user10', ascending=False, inplace=True) 

In [ ]:
# if genre is in genres

In [76]:
train #['Title'].head(num_recs)

,movieId,genres,Title,year_released,userId,rating,timestamp,est_rating_user10
60367,4144,"drama,romance",In the Mood For Love,In the Mood For Love,606.0,4.5,2007-02-13 23:51:00,3.499554
57355,3793,"action,adventure,sci-fi",X-Men,X-Men,332.0,3.5,2012-11-11 22:44:03,3.499554
86412,56367,"comedy,drama,romance",Juno,Juno,414.0,4.0,2008-07-15 18:11:57,3.499554
82571,45672,"adventure,comedy,drama,fantasy,romance",Click,Click,365.0,3.5,2017-03-18 00:33:10,3.499554
60616,4223,"drama,war",Enemy at the Gates,Enemy at the Gates,139.0,4.0,2016-01-27 19:54:21,3.499554
...,...,...,...,...,...,...,...,...
74596,7346,"comedy,romance","Girl Next Door, The","Girl Next Door, The",608.0,4.5,2006-05-16 15:54:24,3.499554
67304,5452,"children,comedy,romance",Look Who's Talking Now,Look Who's Talking Now,414.0,1.0,2002-06-24 14:28:08,3.499554
29707,1304,"action,western",Butch Cassidy and the Sundance Kid,Butch Cassidy and the Sundance Kid,4.0,3.0,1999-12-13 09:47:47,3.499554
61558,4308,"drama,musical,romance",Moulin Rouge,Moulin Rouge,608.0,0.5,2005-05-30 22:06:46,3.499554
